# Transformer

## Config
### Imports

In [1]:
import os
import math
import inspect
from dataclasses import dataclass
import tiktoken

import torch
import torch.nn as nn
from torch.nn import functional as F

## Code

In [2]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
class Tokenizer():
    def __init__(self):
        # Special tokens index
        self.BOS_IDX: int = 100264
        self.EOS_IDX: int = 100265
        self.PAD_IDX: int = 100266

        # Init Tokennizer
        cl100k_base = tiktoken.get_encoding("cl100k_base")
    
        self.encoder = tiktoken.Encoding(
            name="cl100k_bep", # for bos eos pad
            pat_str=cl100k_base._pat_str,
            mergeable_ranks=cl100k_base._mergeable_ranks,
            special_tokens={
                **cl100k_base._special_tokens,
                "<|bos|>": self.BOS_IDX,
                "<|eos|>": self.EOS_IDX,
                "<|pad|>": self.PAD_IDX,
            }
        )
        
    def VocabSize(self):
        return self.encoder.n_vocab
        
    def SequencePadding(self, sequence, max_size = 512):
        """ Method used to add BOS/PAD/EOS special tokens """
        # Checking tensor format
        tensor_sequence = torch.tensor(sequence, dtype=torch.long) if not torch.is_tensor(sequence) else sequence
        # Calculating the current length of the sequence
        sequence_len = tensor_sequence.size()[0]
        # Selecting the size of the current sequence the minimum size is 0 and the maximum size is (max_size - 2) 2 corresponding to bos and eos tags
        cutted_sequence_size = max(0, min(max_size - 2, sequence_len + 2))
        # Cutting the sequence if necessary
        tensor_sequence = tensor_sequence[:cutted_sequence_size]
        # Calculate the padding size
        padding_size = max_size - tensor_sequence.size()[0] - 2 # expected size - current size - (BOS tag + EOS tag) 
        # Adding front tag BOS_IDX
        tensor_sequence = F.pad(tensor_sequence, (1,0), "constant", self.BOS_IDX)
        # Adding padding tag PAD_IDX
        tensor_sequence = F.pad(tensor_sequence, (0,padding_size), "constant", self.PAD_IDX)
        # Adding padding tag EOS_IDX
        tensor_sequence = F.pad(tensor_sequence, (0,1), "constant", self.EOS_IDX)
        return tensor_sequence
    
    def SequenceClearner(self, sequence):
        """ Method used to remove BOS/PAD/EOS special tokens """
        # Checking tensor format
        list_sequence = sequence.tolist() if torch.is_tensor(sequence) else sequence
        def CheckSpecial(number):
            return number not in [self.BOS_IDX, self.EOS_IDX, self.PAD_IDX]
        return list(filter(CheckSpecial, list_sequence))

In [4]:
tokenizer = Tokenizer()
int(tokenizer.VocabSize()/1000)

100

In [5]:
@dataclass
class TransformerConfig:
    block_size: int = 512
    batch_size: int = 24
    vocab_size: int = tokenizer.VocabSize()
    n_layer: int = 6
    n_head: int = 8
    n_embd: int = 512 # From original paper
    dropout: float = 0.1
    bias: bool = False # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster
    # device = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = 'cpu'
    learning_rate = 3e-4
    eval_interval = 1
    eval_iters = 200

In [6]:
from datasets import load_dataset, Dataset

In [7]:
class Data():
    def __init__(self, block_size, batch_size, tokenizer, device):
        self.train_data = load_dataset("wmt14", "fr-en", split="train[:500000]")
        self.val_data = load_dataset("wmt14", "fr-en", split="validation")
        self.test_data = load_dataset("wmt14", "fr-en", split="test")

        self.block_size = block_size
        self.batch_size = batch_size

        self.source_lang = 'en'
        self.target_lang = 'fr'

        self.tokenizer = tokenizer
        self.device = device
    
    def DataSize(self):
        print(f"train_data: {self.train_data.num_rows}")
        print(f"val_data: {self.val_data.num_rows}")
        print(f"test_data: {self.test_data.num_rows}")
        total = self.train_data.num_rows + self.val_data.num_rows + self.test_data.num_rows
        print(f"total: {total}")
        return total

    def PreprocessFunction(self, examples):
        encode = self.tokenizer.encoder.encode
        inputs = [self.tokenizer.SequencePadding(encode(example[self.source_lang]), self.block_size) for example in examples["translation"]]
        targets = [self.tokenizer.SequencePadding(encode(example[self.target_lang]), self.block_size) for example in examples["translation"]]
        return {'inputs': inputs, 'targets': targets}

    # data loading
    def GetBatch(self, split):
        # generate a small batch of data of inputs x and targets y
        if split == 'train':
            data = self.train_data
            data_size = self.train_data.num_rows
        elif split == 'val':
            data = self.val_data
            data_size = self.val_data.num_rows
        else:
            data = self.test_data
            data_size = self.test_data.num_rows
        
        ix = torch.randint(data_size - self.block_size, (min(self.batch_size, data_size),))
        filtered_data = Dataset.from_dict({'translation': [data[i]['translation'] for i in ix.tolist()]})
        
        tokenized_data = filtered_data.map(self.PreprocessFunction, batched=True)
        translation = tokenized_data['translation']
        x = torch.tensor(tokenized_data['inputs'], device=self.device)
        y = torch.tensor(tokenized_data['targets'], device=self.device)
        return x, y, translation

In [8]:
class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

In [9]:
class MultiHeadAttention(nn.Module):
    """ Multi Head Attention without mask """
    def __init__(self, config, is_causal = False, linked = False): # linked represent the blocks getting q and k from encoder
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # INPUTS: query, key, value projections for all heads, but in a batch
        if linked:
            self.c_attn = nn.Linear(3 * config.n_embd, 3 * config.n_embd, bias=config.bias)
        else:
            self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        # OUTPUT: output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # regularization
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.dropout = config.dropout
        # flash attention make GPU go br but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))
        self.is_causal = is_causal

    def forward(self, x, encoder_output=None):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
        
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        if encoder_output is not None:
            x = torch.cat([encoder_output, encoder_output, x], dim=2)
            q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        else:
            q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=self.is_causal)
        else:
            # manual implementation of attention
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1))) # Step 1 & 2: (MatMul) and (Scale)
            if self.is_causal:
                att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1) # Step 3: Softmax
            att = self.attn_dropout(att)
            y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs) # Step 4: MatMul
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side # Step 5: Concatenate

        # output projection
        y = self.resid_dropout(self.c_proj(y)) # Step 6 : Linear
        return y

In [10]:
class FeedForward(nn.Module):
    """ Multi Layer Perceptron (FeedForward) """
    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu    = nn.GELU()
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

In [11]:
class EBlock(nn.Module):
    """ EBlock = (Encoder block) """
    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn = MultiHeadAttention(config, False)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.ffw = FeedForward(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.ffw(self.ln_2(x))
        return x
    

In [12]:
class DBlock(nn.Module):
    # DBlock (Decoder block)
    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn1 = MultiHeadAttention(config, True, False)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn2 = MultiHeadAttention(config, False, True)
        self.ln_3 = LayerNorm(config.n_embd, bias=config.bias)
        self.ffw = FeedForward(config)

    def forward(self, x, encoder_output):
        x = x + self.attn1(self.ln_1(x))
        x = x + self.attn2(self.ln_2(x), encoder_output)
        x = x + self.ffw(self.ln_3(x))
        return x

In [13]:
class Encoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([EBlock(config) for _ in range(config.n_layer)]),
            ln_f = LayerNorm(config.n_embd, bias=config.bias),
        ))
        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters
        print("Number of Encoder parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        """
        Return the number of parameters in the model.
        For non-embedding count (default), the position embeddings get subtracted.
        The token embeddings would too, except due to the parameter sharing these
        params are actually used as weights in the final layer, so we include them.
        """
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb) # Addition of input embd + positional encoding
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = x
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            logits = x
            loss = None

        return logits, loss
    

In [14]:
class Decoder(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([DBlock(config) for _ in range(config.n_layer)]),
            ln_f = LayerNorm(config.n_embd, bias=config.bias),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        # with weight tying when using torch.compile() some warnings get generated:
        # "UserWarning: functional_call was passed multiple values for tied weights.
        # This behavior is deprecated and will be an error in future versions"
        # not 100% sure what this is, so far seems to be harmless. TODO investigate
        self.transformer.wte.weight = self.lm_head.weight # https://paperswithcode.com/method/weight-tying

        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters
        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        """
        Return the number of parameters in the model.
        For non-embedding count (default), the position embeddings get subtracted.
        The token embeddings would too, except due to the parameter sharing these
        params are actually used as weights in the final layer, so we include them.
        """
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb) # Addition of input embd + positional encoding
        for block in self.transformer.h:
            x = block(x, targets)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            print(f"{logits.size()=}")
            print(f"{targets.size()=}")
            print(f"{logits.contiguous().view(-1, logits.size(-1)).shape=}")
            print(f"{targets.contiguous().view(-1, logits.size(-1)).shape=}")
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.contiguous().view(-1, logits.size(-1)), ignore_index=-1)
        else:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            loss = None

        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")

        return optimizer

    def estimate_mfu(self, fwdbwd_per_iter, dt):
        """ estimate model flops utilization (MFU) in units of A100 bfloat16 peak FLOPS """
        # first estimate the number of flops we do per iteration.
        # see PaLM paper Appendix B as ref: https://arxiv.org/abs/2204.02311
        N = self.get_num_params()
        cfg = self.config
        L, H, Q, T = cfg.n_layer, cfg.n_head, cfg.n_embd//cfg.n_head, cfg.block_size
        flops_per_token = 6*N + 12*L*H*Q*T
        flops_per_fwdbwd = flops_per_token * T
        flops_per_iter = flops_per_fwdbwd * fwdbwd_per_iter
        # express our flops throughput as ratio of A100 bfloat16 peak flops
        flops_achieved = flops_per_iter * (1.0/dt) # per second
        flops_promised = 312e12 # A100 GPU bfloat16 peak flops is 312 TFLOPS
        mfu = flops_achieved / flops_promised
        return mfu

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [15]:
class Transformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config
        self.encoder = Encoder(config)
        self.decoder = Decoder(config)

    # def generate_mask(self, src, tgt):
    #     src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
    #     tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
    #     seq_length = tgt.size(1)
    #     nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
    #     tgt_mask = tgt_mask & nopeak_mask
    #     return src_mask, tgt_mask


    def forward(self, src, tgt):
        # src_mask, tgt_mask = self.generate_mask(src, tgt)
        enc_output, _ = self.encoder(src)
        output, loss = self.decoder(tgt, enc_output)
        return output, loss

--------

In [16]:
@torch.no_grad()
def estimate_loss(dataset, config):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(config.eval_iters)
        for k in range(config.eval_iters):
            X, Y, _ = dataset.GetBatch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [17]:
config = TransformerConfig()
model = Transformer(config)
dataset = Data(config.block_size, config.batch_size, tokenizer, config.device)
dataset.DataSize()

Number of Encoder parameters: 70.22M
number of parameters: 85.95M


Found cached dataset wmt14 (C:/Users/thiba/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)
Found cached dataset wmt14 (C:/Users/thiba/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)
Found cached dataset wmt14 (C:/Users/thiba/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)


train_data: 500000
val_data: 3000
test_data: 3003
total: 506003


506003

In [18]:
# create a PyTorch optimizer
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate, betas=(0.9, 0.98), eps=1e-9)

model.train()
model = model.to(config.device)

In [19]:
max_iters = 5

In [23]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    # if iter % config.eval_interval == 0:
    #     losses = estimate_loss(dataset, config)
    #     print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb, _ = dataset.GetBatch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

KeyboardInterrupt: 